In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import asf_search as asf

In [3]:
import asf_search as asf
import logging

# asf.constants.INTERNAL.CMR_TIMEOUT = 90
# asf.ASF_LOGGER.setLevel(logging.DEBUG)

results = asf.product_search('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252-SLC')
results = asf.product_search('S1_181296_IW1_20220219T125501_VV_10AF-BURST')
reference = results[0]

args = asf.ASFSearchOptions(
    **{"start": '2022-02-10', "end": '2022-07-01'}
)
s = reference.stack(args)

### Create a Pair object from 2 products

In [4]:
pair = asf.Pair(reference, s[2])

print(f"pair.ref.properties['sceneName']: {pair.ref.properties['sceneName']}")
print(f"pair.ref_date: {pair.ref_date}\n")

print(f"pair.sec.properties['sceneName']: {pair.sec.properties['sceneName']}")
print(f"pair.sec_date: {pair.sec_date}\n")

print(f"pair.temporal: {pair.temporal}")
print(f"pair.perpendicular: {pair.perpendicular}")

pair.ref.properties['sceneName']: S1_181296_IW1_20220219T125501_VV_10AF-BURST
pair.ref_date: 2022-02-19

pair.sec.properties['sceneName']: S1_181296_IW1_20220315T125501_VV_6DCD-BURST
pair.sec_date: 2022-03-15

pair.temporal: 24 days, 0:00:00
pair.perpendicular: -104


### Check the estimated coherence of the pair

In [5]:
pair.estimate_s1_mean_coherence()

43.525541125541125

### Create a new pair with a long temporal baseline

In [6]:
long_pair = pair = asf.Pair(reference, s[10])
print(f"long_pair.temporal: {long_pair.temporal}")

long_pair.temporal: 120 days, 0:00:00


### Since the temporal baseline is greater than 48, an exception is raised when checking coherence

In [7]:
long_pair.estimate_s1_mean_coherence()

Exception: Coherence dataset includes temporal baselines up to 48 days.
            Temporal baseline: 120 days

### Demonstrate pair equivalence

Two pairs are equal if they both share the same reference and secondary dates.

In [8]:
from copy import deepcopy

a = deepcopy(pair)
b = deepcopy(pair)
c = asf.Pair(reference, s[1])
print(f"a.ref_date: {a.ref_date}, a.sec_date: {a.sec_date}")
print(f"b.ref_date: {b.ref_date}, b.sec_date: {b.sec_date}")
print(f"c.ref_date: {c.ref_date}, c.sec_date: {c.sec_date}")
print(f"a == b: {a == b}")
print(f"a == c: {a == c}")

a.ref_date: 2022-02-19, a.sec_date: 2022-06-19
b.ref_date: 2022-02-19, b.sec_date: 2022-06-19
c.ref_date: 2022-02-19, c.sec_date: 2022-03-03
a == b: True
a == c: False


# Stack

### Create a Stack from the reference scene object instantiated above

Stack accepts ASFSearchOptions, which are used in a stack_from_product search

In [9]:
args = asf.ASFSearchOptions(
    **{"start": '2022-01-01', "end": '2022-03-02'}
)

stack = asf.Stack(reference, opts=args)
stack.full_stack

{(datetime.date(2022, 1, 2),
  datetime.date(2022, 1, 14)): Pair(2022-01-02, 2022-01-14),
 (datetime.date(2022, 1, 2),
  datetime.date(2022, 1, 26)): Pair(2022-01-02, 2022-01-26),
 (datetime.date(2022, 1, 2),
  datetime.date(2022, 2, 7)): Pair(2022-01-02, 2022-02-07),
 (datetime.date(2022, 1, 2),
  datetime.date(2022, 2, 19)): Pair(2022-01-02, 2022-02-19),
 (datetime.date(2022, 1, 14),
  datetime.date(2022, 1, 26)): Pair(2022-01-14, 2022-01-26),
 (datetime.date(2022, 1, 14),
  datetime.date(2022, 2, 7)): Pair(2022-01-14, 2022-02-07),
 (datetime.date(2022, 1, 14),
  datetime.date(2022, 2, 19)): Pair(2022-01-14, 2022-02-19),
 (datetime.date(2022, 1, 26),
  datetime.date(2022, 2, 7)): Pair(2022-01-26, 2022-02-07),
 (datetime.date(2022, 1, 26),
  datetime.date(2022, 2, 19)): Pair(2022-01-26, 2022-02-19),
 (datetime.date(2022, 2, 7),
  datetime.date(2022, 2, 19)): Pair(2022-02-07, 2022-02-19)}

### Remove pairs from the full stack, identifying them by ref and sec dates

Both full_stack and subset_stack contain the same Pair objects, so keeping both stacks does not eat up a ton of extra memory

In [10]:
import numpy as np
import pandas as pd
import datetime

stack.remove_pairs([
    (datetime.date(2022, 1, 10), datetime.date(2022, 1, 22)), # datetime.date
    (pd.to_datetime("2022-1-10"), pd.to_datetime("2022- 2-3")), # pandas.Timestamp
    ("2022-01-10", "2022-02-15"), # isoformat string
    (np.datetime64("2022-01-10"), np.datetime64("2022-02-27")) # numpy.datetime64
])

stack.remove_list

/Users/alexlewandowski/Documents/Discovery-asf_search/asf_search/Stack.py:91: UserWarning: warning: (datetime.date(2022, 1, 10), datetime.date(2022, 1, 22)) is not in Stack.full_stack
  warnings.warn(
/Users/alexlewandowski/Documents/Discovery-asf_search/asf_search/Stack.py:91: UserWarning: warning: (datetime.date(2022, 1, 10), datetime.date(2022, 2, 3)) is not in Stack.full_stack
  warnings.warn(
/Users/alexlewandowski/Documents/Discovery-asf_search/asf_search/Stack.py:91: UserWarning: warning: (datetime.date(2022, 1, 10), datetime.date(2022, 2, 15)) is not in Stack.full_stack
  warnings.warn(
/Users/alexlewandowski/Documents/Discovery-asf_search/asf_search/Stack.py:91: UserWarning: warning: (datetime.date(2022, 1, 10), datetime.date(2022, 2, 27)) is not in Stack.full_stack
  warnings.warn(


[]

### Look at subset_stack, after having removed some pairs from full_stack

In [11]:
stack.subset_stack

{(datetime.date(2022, 1, 2),
  datetime.date(2022, 1, 14)): Pair(2022-01-02, 2022-01-14),
 (datetime.date(2022, 1, 2),
  datetime.date(2022, 1, 26)): Pair(2022-01-02, 2022-01-26),
 (datetime.date(2022, 1, 2),
  datetime.date(2022, 2, 7)): Pair(2022-01-02, 2022-02-07),
 (datetime.date(2022, 1, 2),
  datetime.date(2022, 2, 19)): Pair(2022-01-02, 2022-02-19),
 (datetime.date(2022, 1, 14),
  datetime.date(2022, 1, 26)): Pair(2022-01-14, 2022-01-26),
 (datetime.date(2022, 1, 14),
  datetime.date(2022, 2, 7)): Pair(2022-01-14, 2022-02-07),
 (datetime.date(2022, 1, 14),
  datetime.date(2022, 2, 19)): Pair(2022-01-14, 2022-02-19),
 (datetime.date(2022, 1, 26),
  datetime.date(2022, 2, 7)): Pair(2022-01-26, 2022-02-07),
 (datetime.date(2022, 1, 26),
  datetime.date(2022, 2, 19)): Pair(2022-01-26, 2022-02-19),
 (datetime.date(2022, 2, 7),
  datetime.date(2022, 2, 19)): Pair(2022-02-07, 2022-02-19)}

### Look at connected_substacks

If subset_stack is disconnected, its component stacks are stored in connected_substacks.

Since we have not yet done anything to disconnect the subset_stack, this list contains only one item.

When we use the Network class to make SBAS stacks, we may have multiple connected substacks.



In [12]:
stack.connected_substacks

[{(datetime.date(2022, 1, 2),
   datetime.date(2022, 1, 14)): Pair(2022-01-02, 2022-01-14),
  (datetime.date(2022, 1, 2),
   datetime.date(2022, 1, 26)): Pair(2022-01-02, 2022-01-26),
  (datetime.date(2022, 1, 2),
   datetime.date(2022, 2, 7)): Pair(2022-01-02, 2022-02-07),
  (datetime.date(2022, 1, 2),
   datetime.date(2022, 2, 19)): Pair(2022-01-02, 2022-02-19),
  (datetime.date(2022, 1, 14),
   datetime.date(2022, 1, 26)): Pair(2022-01-14, 2022-01-26),
  (datetime.date(2022, 1, 14),
   datetime.date(2022, 2, 7)): Pair(2022-01-14, 2022-02-07),
  (datetime.date(2022, 1, 14),
   datetime.date(2022, 2, 19)): Pair(2022-01-14, 2022-02-19),
  (datetime.date(2022, 1, 26),
   datetime.date(2022, 2, 7)): Pair(2022-01-26, 2022-02-07),
  (datetime.date(2022, 1, 26),
   datetime.date(2022, 2, 19)): Pair(2022-01-26, 2022-02-19),
  (datetime.date(2022, 2, 7),
   datetime.date(2022, 2, 19)): Pair(2022-02-07, 2022-02-19)}]

## Network

Create a Network from the same reference scene. 

Attempts to create a connected, seasonal SBAS stack based on:
- georeference scene
- season
- start date
- end date
- perpendicular baseline
- temporal baseline
  

In [13]:
import pandas as pd
from datetime import datetime

def get_julian_season(self) -> tuple[int,int]:
    season_start_ts = pd.Timestamp(
        datetime.strptime(f"{season[0]}-0001", "%m-%d-%Y"), tz="UTC"
        )
    season_start_day = season_start_ts.timetuple().tm_yday
    season_end_ts = pd.Timestamp(
        datetime.strptime(f"{season[1]}-0001", "%m-%d-%Y"), tz="UTC"
    )
    season_end_day = season_end_ts.timetuple().tm_yday
    return (season_start_day, season_end_day)

season = ("1-1", "6-25")

opts = asf.ASFSearchOptions(
    **{
        "start": '2014-01-01', 
        "end": '2025-10-02', 
        "season": get_julian_season(season)
        }
)

network = asf.Network(
    geo_reference = reference,
    perp_baseline=200, 
    inseason_temporal_baseline=12,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

print(f"len(network.full_stack): {len(network.full_stack)}")
print(f"len(network.remove_list): {len(network.remove_list)}")
print(f"len(network.subset_stack): {len(network.subset_stack)}")
print(f"len(network.connected_substacks): {len(network.connected_substacks)}")

len(network.full_stack): 3010
len(network.remove_list): 2848
len(network.subset_stack): 162
len(network.connected_substacks): 6


In [14]:
network.plot()

### Create an updated network with shorter baselines

In [15]:
network = asf.Network(
    reference,
    perp_baseline=50, 
    inseason_temporal_baseline=12,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

network.plot()

### Create an updated network with longer baselines

In [16]:
network = asf.Network(
    reference,
    perp_baseline=200, 
    inseason_temporal_baseline=24,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

network.plot()

### Plot full_stack

You can pass an any member stack (or list of member stacks) to the plot function.

In [17]:
network.plot(stack_dict=network.full_stack)

### Call `Stack.get_insar_pair_ids()` on the largest connected substack

Provides the data structure you want when ordering GAMMA or single-burst interferograms from HyP3

In [18]:
network.get_insar_pair_ids()

[('S1_181296_IW1_20150103T125420_VV_59B9-BURST',
  'S1_181296_IW1_20150127T125419_VV_8967-BURST'),
 ('S1_181296_IW1_20150127T125419_VV_8967-BURST',
  'S1_181296_IW1_20150220T125419_VV_3017-BURST'),
 ('S1_181296_IW1_20150220T125419_VV_3017-BURST',
  'S1_181296_IW1_20150304T125419_VV_1C18-BURST'),
 ('S1_181296_IW1_20150220T125419_VV_3017-BURST',
  'S1_181296_IW1_20160203T125425_VV_0C5C-BURST'),
 ('S1_181296_IW1_20150220T125419_VV_3017-BURST',
  'S1_181296_IW1_20160227T125425_VV_30C6-BURST'),
 ('S1_181296_IW1_20150220T125419_VV_3017-BURST',
  'S1_181296_IW1_20170128T125431_VV_6B1F-BURST'),
 ('S1_181296_IW1_20150220T125419_VV_3017-BURST',
  'S1_181296_IW1_20170209T125431_VV_F6E8-BURST'),
 ('S1_181296_IW1_20150220T125419_VV_3017-BURST',
  'S1_181296_IW1_20170221T125431_VV_4D57-BURST'),
 ('S1_181296_IW1_20150304T125419_VV_1C18-BURST',
  'S1_181296_IW1_20150328T125419_VV_08AA-BURST'),
 ('S1_181296_IW1_20150304T125419_VV_1C18-BURST',
  'S1_181296_IW1_20160227T125425_VV_30C6-BURST'),
 ('S1_1812

### `Stack.get_insar_pair_ids()` defaults to using the largest connected substack, but you can pass it any member stack

In [19]:
print(len(network.get_insar_pair_ids()))
print(len(network.get_insar_pair_ids(network.full_stack)))

411
3094


### MultiBurst Class

In [22]:
# 3x3 adjactent bursts
# Should pass

multiburst_dict = {
    "173_370305": ("IW1", "IW2", "IW3"),
    "173_370306": ("IW1", "IW2", "IW3"),
    "173_370307": ("IW1", "IW2", "IW3")
}

asf.MultiBurst(multiburst_dict)

In [23]:
# 3x3 adjactent bursts, but crosses orbital paths
# should fail

multiburst_dict = {
    "173_370305": ("IW1", "IW2", "IW3"),
    "173_370306": ("IW1", "IW2", "IW3"),
    "100_213509": ("IW1", "IW2", "IW3")
}

asf.MultiBurst(multiburst_dict)

Exception: All bursts must belong to the same orbital path.
multiburst_dict contains burst with the following paths: {'173', '100'}

In [24]:
# Contains disconnected bursts and a hole
# Should fail

multiburst_dict = {
    "173_370305": ("IW1", "IW2", "IW3"),
    "173_370306": ("IW1", "IW2", "IW3"),
    "173_370307": ("IW1", ),
    "173_370308": ( "IW3",),
    "173_370309": ("IW1", "IW2", "IW3"),
    "173_370310": ("IW1", "IW3"),
    "173_370311": ("IW1", "IW2", "IW3")
}

mb = asf.MultiBurst(multiburst_dict)

Exception: Multiburst collections must be comprised of a single connected component and have no holes.
Connected Components: 2, Holes: 1

In [25]:
# Diagonal line of bursts
# Should pass

multiburst_dict = {
    "173_370305": ("IW1",),
    "173_370306": ("IW2",),
    "173_370307": ("IW3",),
}

asf.MultiBurst(multiburst_dict)


In [26]:
# "Vertical" zigzag line of bursts
# Should pass (but will currently fail when ordering from HyP3)

multiburst_dict = {
    "173_370305": ["IW1"],
    "173_370306": ["IW2"],
    "173_370307": ["IW1"],
}

asf.MultiBurst(multiburst_dict)

In [27]:
# "Horizontal" zigzag line of bursts
# Should pass (but will currently fail when ordering from HyP3)

multiburst_dict = {
    "173_370305": ("IW1", "IW3"),
    "173_370306": ("IW2",)
}

asf.MultiBurst(multiburst_dict)


In [28]:

# Intersects the Antimeridan
# Should fail

multiburst_dict = {
    "001_000664": ("IW1", "IW2", "IW3"),
    "001_000665": ("IW1", "IW2", "IW3"),
    "001_000666": ("IW1", "IW2", "IW3")
}

asf.MultiBurst(multiburst_dict)

Exception: No bursts can intersect the Antimeridian

### Multiburst Network

In [29]:
multiburst_dict = {
    "085_181296": ("IW1", "IW2", "IW3"),
    "085_181297": ("IW1", "IW2", "IW3"),
    "085_181298": ("IW1", "IW2", "IW3")
}

multiburst_dict = {
    "085_181285": ("IW1", "IW2", ),
    "085_181286": ("IW1", "IW2", ),
    "085_181287": ("IW1", "IW2", )
}

# # will generate CMR search error
# multiburst_dict = {
#     "093_197801": ("IW2", "IW3", ),
# }

multiburst = asf.MultiBurst(multiburst_dict)

In [30]:
import asf_search as asf
import pandas as pd
from datetime import datetime

def get_julian_season(self) -> tuple[int,int]:
    season_start_ts = pd.Timestamp(
        datetime.strptime(f"{season[0]}-0001", "%m-%d-%Y"), tz="UTC"
        )
    season_start_day = season_start_ts.timetuple().tm_yday
    season_end_ts = pd.Timestamp(
        datetime.strptime(f"{season[1]}-0001", "%m-%d-%Y"), tz="UTC"
    )
    season_end_day = season_end_ts.timetuple().tm_yday
    return (season_start_day, season_end_day)

season = ("1-1", "6-25")

opts = asf.ASFSearchOptions(
    **{
        # "start": '2015-01-01', 
        # "end": '2016-05-02', 
        "start": '2022-01-01',
        "end": '2025-05-02', 
        "season": get_julian_season(season)
        }
)

network = asf.Network(
    multiburst = multiburst,
    perp_baseline=400, 
    inseason_temporal_baseline=24,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

print(f"len(network.full_stack): {len(network.full_stack)}")
print(f"len(network.remove_list): {len(network.remove_list)}")
print(f"len(network.subset_stack): {len(network.subset_stack)}")
print(f"len(network.connected_substacks): {len(network.connected_substacks)}")

len(network.full_stack): 1077
len(network.remove_list): 908
len(network.subset_stack): 169
len(network.connected_substacks): 1


In [31]:
network.plot()

In [32]:
for n in network.additional_multiburst_networks:
    print(len(max(n.connected_substacks, key=lambda s: len(s))))

169
169
169
169
169


In [36]:
refs, secs = network.get_multi_burst_pair_ids()

print(len(refs))
print(len(secs))

print(refs[:10])
print(secs[:10])

1014
1014
['S1_181283_IW1_20220102T125427_VV_C11B-BURST', 'S1_181283_IW1_20220102T125427_VV_C11B-BURST', 'S1_181283_IW1_20220114T125427_VV_6CB7-BURST', 'S1_181283_IW1_20220114T125427_VV_6CB7-BURST', 'S1_181283_IW1_20220126T125426_VV_4CA7-BURST', 'S1_181283_IW1_20220126T125426_VV_4CA7-BURST', 'S1_181283_IW1_20220207T125426_VV_978C-BURST', 'S1_181283_IW1_20220207T125426_VV_978C-BURST', 'S1_181283_IW1_20220207T125426_VV_978C-BURST', 'S1_181283_IW1_20220207T125426_VV_978C-BURST']
['S1_181283_IW1_20220114T125427_VV_6CB7-BURST', 'S1_181283_IW1_20220126T125426_VV_4CA7-BURST', 'S1_181283_IW1_20220126T125426_VV_4CA7-BURST', 'S1_181283_IW1_20220207T125426_VV_978C-BURST', 'S1_181283_IW1_20220207T125426_VV_978C-BURST', 'S1_181283_IW1_20220219T125426_VV_4808-BURST', 'S1_181283_IW1_20220219T125426_VV_4808-BURST', 'S1_181283_IW1_20220303T125425_VV_B722-BURST', 'S1_181283_IW1_20230121T125432_VV_6BE2-BURST', 'S1_181283_IW1_20230202T125432_VV_F1EE-BURST']
